In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


### Preparing dataset for model training
Defining a Dataset class to load the tensor label pairs and dataloader to feed the training in batches

In [3]:
from torch.utils.data import Dataset
class GTSRBImageDataset(Dataset):
    def __init__(self, label_file, file_dir = "", transform = None):
        self.df = pd.read_csv(label_file)
        self.labels = self.df["Labels"]
        self.file_names = self.df["Path"]
        self.file_dir = file_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        file_path = os.path.join(self.file_dir, self.file_names[index])
        img = cv2.imread(file_path)
        label = self.labels[index]
        if self.transform:
            img = self.transform(img)
        return img, label

In [4]:
from torchvision.transforms import ToTensor
transform = ToTensor()
dataset = GTSRBImageDataset("labels.csv", transform=transform)

In [5]:
from torch.utils.data import random_split
train, test = random_split(dataset, [0.7,0.3])
training_loader = DataLoader(train, batch_size=32, shuffle=True)
test_loader = DataLoader(test, batch_size=32, shuffle=True)

### Test Model
Setting up a simple CNN model to test training on the datset

In [6]:
import torch.nn.functional as F


class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        
        x = torch.flatten(x, start_dim=1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x


### Helper functions
Preparing helper functions for training and validation

In [7]:
from tqdm import tqdm
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
from torchmetrics import Accuracy
def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None, num_classes=10):
    model.to(device)
    loss_fn.to(device)
    model.train()
    
    loss_train = AverageMeter()
    acc_train = Accuracy(task="multiclass", num_classes=num_classes).to(device)
    acc_train.reset()

    with tqdm(train_loader, unit="batch") as tepoch:
        if epoch is not None:
            tepoch.set_description(f"Epoch {epoch}")

        for inputs, targets in tepoch:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad() 
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)

            loss.backward()
            optimizer.step()

            loss_train.update(loss.item())
            acc_train.update(outputs, targets.int())

            tepoch.set_postfix(loss=loss_train.avg)

    final_accuracy = acc_train.compute().item()
    return model, loss_train.avg, final_accuracy

import sklearn.metrics as metrics
def validation(model, test_loader, loss_fn):
  model.eval()
  with torch.no_grad():
    loss_valid = AverageMeter()
    acc_valid = Accuracy(task="multiclass", num_classes=43).to(device)
    predictions, labels = [], []
    for i, (inputs, targets) in enumerate(test_loader):
      inputs = inputs.to(device)
      targets = targets.to(device)

      outputs = model(inputs)
      loss = loss_fn(outputs, targets)

      loss_valid.update(loss.item())
      acc_valid(outputs, targets.int())
      predictions.append(torch.max(outputs.data, 1)[1].cpu().numpy())
      labels.append(targets.cpu().numpy())
    predictions = np.concatenate(predictions)
    labels = np.concatenate(labels)
    confusion = metrics.confusion_matrix(labels,predictions)
  return loss_valid.avg, acc_valid.compute().item(), confusion

### Training the sample model

In [24]:
net = SimpleCNN(43)
print(net)
def train_model(model, epoch_amount, loader):
    model.to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for epoch in range(epoch_amount):
        model, loss, acc = train_one_epoch(model, loader, criterion, optimizer, num_classes=43, epoch=epoch)
    print(f"Model accuracy {acc}, Loss {loss}")
train_model(net, 20, training_loader)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=8192, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=43, bias=True)
)


Epoch 0:   0%|          | 0/858 [00:00<?, ?batch/s]

Epoch 19: 100%|██████████| 858/858 [00:09<00:00, 90.10batch/s, loss=0.00228]

Model accuracy 1.0, Loss 0.0022810054731979695


In [26]:
test_loss, test_acc, confusion = validation(net, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9955790042877197, test loss 0.015844079969571358
[[ 58   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 674   1   1   0   1   0   0   0   0   0   0   0   0   0   0   0   1
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0 664   0   0   1   0   0   1   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 415   0   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0 601   0   0   1   2   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   3   0 533   0   0   0   

In [27]:
total_params = sum(p.numel() for p in net.parameters())
print(total_params)

2202155


In [28]:
torch.save(net, "model_CNN.pt")

In [29]:
CNN_model = torch.load("model_CNN.pt", weights_only=False)

swedish_dataset = GTSRBImageDataset("swedish_labels.csv", transform=transform)
swedish_train, swedish_test = random_split(swedish_dataset, [0.7,0.3])
swedish_training_loader = DataLoader(swedish_train, batch_size=32, shuffle=True)
swedish_test_loader = DataLoader(swedish_test, batch_size=32, shuffle=True)

In [30]:
test_loss, test_acc, confusion = validation(CNN_model, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.40831294655799866, test loss 3.3800436716813307
[[ 6  0  4  0  2  3  0  0  2  0  0  0  0  0  0  0  0  0  0  2  0  0  0  0
   0  1  0  0  0  0  0  0  0  0]
 [ 3  1  1  0  2  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  1
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  2  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 28  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  

In [31]:
train_model(CNN_model,10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:00<00:00, 92.57batch/s, loss=0.028] 

Model accuracy 0.9968652129173279, Loss 0.027965453049788872


In [32]:
test_loss, test_acc, confusion = validation(CNN_model, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9315403699874878, test loss 0.2749882409922205
[[18  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  6  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 29  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  0  0  0 50  2  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0 11  0  0  0  0  0  0  1  0  0  0  0  1]
 [ 0  0  0  1  0  0 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0 28  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 1  0  0  0  0  0  0  0  0  0 86  1  2  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  1 31  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  1  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  1  0  0  0  0 50  0  1]
 [ 0  0  0  0  0  0  0  0

In [34]:
import torchvision.models as models
class ModifiedResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedResNet, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.model.maxpool = nn.Identity()
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.model(x)
model_Resnet = ModifiedResNet(num_classes=43)


/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/img/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 102MB/s] 


In [35]:
train_model(model_Resnet, 20, training_loader)

Epoch 19: 100%|██████████| 858/858 [01:02<00:00, 13.70batch/s, loss=0.000993]

Model accuracy 1.0, Loss 0.0009928288882992302


In [38]:
torch.save(model_Resnet, "model_Resnet.pt")

In [40]:
test_loss, test_acc, confusion = validation(model_Resnet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9995748996734619, test loss 0.0016925141946914032
[[ 58   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 677   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0 666   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 415   0   1   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0 604   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0 538   0   0   0  

In [41]:
test_loss, test_acc, confusion = validation(model_Resnet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.5647921562194824, test loss 1.6480753330083995
[[16  2  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 1  7  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0 29  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0  1 51  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 12  0  0  0  0  0  0  0  0  0  0  1  0  0  0  1
   0  0  0  0  0]
 [ 0  0  0  3  3  0  0  0  0 10  1  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0]
 [ 0  0  0  0  0 

In [42]:
transfer_resnet = torch.load("model_Resnet.pt", weights_only=False)

In [43]:
train_model(transfer_resnet, 10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:02<00:00, 13.77batch/s, loss=0.0141]

Model accuracy 0.9989550709724426, Loss 0.014121472494055827


In [44]:
test_loss, test_acc, confusion = validation(transfer_resnet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9061384201049805, test loss 0.2813124354199871
[[ 50   0   0   0   8   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 657   1   0   7   1   0   6   1   0   0   0   0   0   0   2   0   2
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   1   0   0]
 [  0   0 650   0   4   0   0   1   3   0   0   0   0   0   0   2   0   1
    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0
    0   0   3   0   1   0   0]
 [  0   4   2 397   0   0   0   0   0   0   0   0   0   1   0   9   0   0
    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   1
    0   0   1   0   0   0   0]
 [  0   0   0   0 593   0   0   0   7   0   0   0   0   0   0   4   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0  73   9  12   1 400   0  10  21   0 

In [45]:
test_loss, test_acc, confusion = validation(transfer_resnet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9779950976371765, test loss 0.11002011063437049
[[19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  7  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 89  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 33  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  1  0  0  0  0 51  0  0]
 [ 0  0  0  0  0  0  0  

In [48]:
class ModifiedMobileNetV2(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedMobileNetV2, self).__init__()
        self.model = models.mobilenet_v2(pretrained=True)
        self.model.features[0][0] = nn.Conv2d(
            in_channels=3, 
            out_channels=32, 
            kernel_size=3, 
            stride=1, 
            padding=1, 
            bias=False
        )
        self.model.classifier[1] = nn.Linear(self.model.last_channel, num_classes)

    def forward(self, x):
        return self.model(x)

In [49]:
model_mobilenet = ModifiedMobileNetV2(43)

/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [50]:
train_model(model_mobilenet, 20, training_loader)

Epoch 19: 100%|██████████| 858/858 [00:28<00:00, 29.76batch/s, loss=0.000783]

Model accuracy 0.9999635815620422, Loss 0.0007828021950035797


In [51]:
torch.save(model_mobilenet, "model_Mobilenet.pt")

In [52]:
test_loss, test_acc, confusion = validation(model_mobilenet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9989797472953796, test loss 0.0029588729388132892
[[ 58   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 678   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0 666   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 416   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0 604   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   1   0   0   0 537   0   0   0  

In [53]:
test_loss, test_acc, confusion = validation(model_mobilenet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.567237138748169, test loss 2.0667291421156664
[[16  1  1  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  7  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 28  0  0  0  2  0  0  0  0 

In [54]:
transfer_mobilenet = torch.load("model_Mobilenet.pt", weights_only=False)

In [55]:
train_model(transfer_mobilenet, 10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:01<00:00, 28.18batch/s, loss=0.0114] 

Model accuracy 0.9979101419448853, Loss 0.01144852238163973


In [56]:
test_loss, test_acc, confusion = validation(transfer_mobilenet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.822989284992218, test loss 0.7226567385799211
[[ 44   0   0   0  14   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 666   1   0   8   0   0   1   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   1   0   0]
 [  0   5 652   1   3   0   0   0   0   0   0   0   0   0   0   3   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   2   0   0   0   0]
 [  0  17  15 362   0   0   0   0   0   0   0   0   0   0   3  15   0   0
    0   0   0   0   0   0   0   0   0   4   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   1   0   0 600   0   0   0   0   0   0   0   0   0   0   2   0   0
    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 110  34   4   2 332   0  10   0   0  

In [57]:
test_loss, test_acc, confusion = validation(transfer_mobilenet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9853300452232361, test loss 0.10484904478877209
[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 89  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1 33  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  9  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 52  0  0]
 [ 0  0  0  0  0  0  0  

In [61]:
class ModifiedSqueezeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ModifiedSqueezeNet, self).__init__()
        self.model = models.squeezenet1_0(pretrained=True)
        self.model.features[0] = nn.Conv2d(
            in_channels=3,
            out_channels=96,
            kernel_size=3,
            stride=1,
            padding=1
        )
        self.model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=1)

    def forward(self, x):
        return self.model(x)

In [62]:
model_SqueezeNet = ModifiedSqueezeNet(43)

/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /home/img/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:00<00:00, 42.9MB/s]


In [63]:
train_model(model_SqueezeNet, 20, training_loader)

Epoch 0:   0%|          | 0/858 [00:00<?, ?batch/s]

Epoch 19: 100%|██████████| 858/858 [00:20<00:00, 41.29batch/s, loss=0.00582]

Model accuracy 0.9983604550361633, Loss 0.005815541728075244


In [64]:
test_loss, test_acc, confusion = validation(model_SqueezeNet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9980445504188538, test loss 0.007756903604951172
[[ 58   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 675   0   0   1   1   0   0   1   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   1 665   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0 414   0   2   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0 604   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0   0   0   0   0 537   0   1   0   

In [65]:
test_loss, test_acc, confusion = validation(model_SqueezeNet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.5061124563217163, test loss 4.571679041935847
[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 2  9  2  0  0  0  5  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  6  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  0  0  0  0  0  0  0  9 34  4  0  5  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0]
 [ 0  1  0  0  0  0  0  0  1  0 10  0  0  1  0  

In [66]:
torch.save(model_SqueezeNet, "model_Squeezenet.pt")

In [67]:
transfer_squeezenet = torch.load("model_Squeezenet.pt", weights_only=False)

In [68]:
train_model(transfer_squeezenet, 10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:00<00:00, 40.44batch/s, loss=0.0346]

Model accuracy 0.9874607920646667, Loss 0.03459173126223807


In [69]:
test_loss, test_acc, confusion = validation(transfer_squeezenet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.7250467538833618, test loss 1.1316048781913908
[[  1   1   0   0  56   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 645   0   0   7   0   0   0   0   0   0   0   0   0   1  24   0   0
    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0  25 608   0   1   0   0   0   0   0   0   0   0   0   1  26   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   3   0   0
    0   0   2   0   0   0   0]
 [  0  19  14 305   0   0   0   1   0   1   0   0   1   0   1  45   0   0
    0   0   0   0   0   0   0   0   0   0   1   0   0   0   0   2   0   1
    0   0   2   0  23   0   0]
 [  0   0   0   0 582   0   0   0   0   0   0   0   0   0   0  22   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0 287   7   0   0 134   0   9   0   0 

In [70]:
test_loss, test_acc, confusion = validation(transfer_squeezenet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9584352374076843, test loss 0.279107415690445
[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 29  0  0  0  0  0  0  0  0  0  0  0  1  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 13  0  0  0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  1 16  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0  0 28  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 89  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2 31  0  0  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  2  0 13  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0  0]
 [ 0  0  0  1  1  0  0  0  0  0  0  0  0  0  0 50  0  1]
 [ 0  0  0  0  0  0  0  0 

In [71]:
non_transfer_cnn = SimpleCNN(43)

In [72]:
train_model(non_transfer_cnn, 10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:00<00:00, 90.07batch/s, loss=0.183]

Model accuracy 0.9717868566513062, Loss 0.18266847605506578


In [73]:
test_loss, test_acc, confusion = validation(non_transfer_cnn, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.11885733902454376, test loss 4.738511174917221
[[  0  13   3   0   0   0   0   0   0   0   0   0   0   0  23   5   0   4
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  10   0   0]
 [  0  85  35   0  13   0   0   0   0   0   0   0  17   0 241  92   0  54
    0   0   0   0   0   0   0   0   0  31  11   0   0   0   0   1   0   0
    0   0  33   0  65   0   0]
 [  0  13  34   0   3   0   0   0   0   0   0   0  20   0 320  86   0  42
    0   0   0   0   0   0   0   0   0  20   1   0   0   0   0   4   0   0
    0   0  43   0  80   0   0]
 [  0  11   3   0   0   0   0   0   0   0   0   0  16   1 237  37   0  39
    0   0   0   0   0   0   0   0   0  26   5   0   0   0   0  12   0   0
    0   0   8   0  21   0   0]
 [  0  14  22   0   2   0   0   0   0   0   0   0  28   0 338  55   0  30
    0   0   0   0   0   0   0   0   0  89   0   0   0   0   0   2   0   0
    0   0  14   0  10   0   0]
 [  0   4  10   0   0   0   0   0   0   0 

In [74]:
test_loss, test_acc, confusion = validation(non_transfer_cnn, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9144254326820374, test loss 0.40635058971551746
[[19  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 0  7  0  0  1  0  0  0  0  1  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 29  0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 51  0  1  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  2 10  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 16  1  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 84  1  2  1  0  2  0  0]
 [ 0  0  0  0  0  0  0  1  0  0  2 31  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0 14  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  1  0  0  8  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  1  1  0  0  0  0  0  4  0  0  0  0 46  0  1]
 [ 0  0  0  0  0  0  0  

In [75]:
non_transfer_Resnet = ModifiedResNet(num_classes=43)

/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [76]:
train_model(non_transfer_Resnet, 10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:02<00:00, 13.33batch/s, loss=0.0885]

Model accuracy 0.9895506501197815, Loss 0.08851141383250555


In [77]:
test_loss, test_acc, confusion = validation(non_transfer_Resnet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.1664682924747467, test loss 4.552958864232768
[[  0  16   1   0   0   0   0   0   0   0   0   0   0   0  25   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  16   0   0]
 [  0 236   4   0   1   0   0   0   0   0   0   0   0   2  52  25   0  40
    0   0   0   0   0   0   0   0   0   1   0   7   0   0   0   7   0   0
    0   0   1   0 302   0   0]
 [  0 181   7   0   2   0   0   0   0   0   0   0   0   2 122  37   0  45
    0   0   0   0   0   0   0   0   0  13   0   7   0   0   0  26   0   0
    0   0   4   0 220   0   0]
 [  0 188   3   0   0   0   0   0   0   0   0   0   1   6  17  27   0  23
    0   0   0   0   0   0   0   0   0  20   1  14   0   0   0  59   0   0
    0   0   0   0  57   0   0]
 [  0 116   0   0   1   0   0   0   0   0   0   0   0   3 113  49   0  19
    0   0   0   0   0   0   0   0   0  38   0  28   0   0   0  66   0   0
    0   0  11   0 160   0   0]
 [  0 179   8   0   0   0   0   0   0   0  

In [78]:
test_loss, test_acc, confusion = validation(non_transfer_Resnet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9584352374076843, test loss 0.2188305496596373
[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  9  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 88  1  1  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2 32  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  8  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  0  0]
 [ 0  0  0  0  1  0  0  1  0  0  1  0  0  0  0 50  0  0]
 [ 0  0  0  0  0  0  0  0

In [79]:
non_transfer_mobilenet = ModifiedMobileNetV2(43)

/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [80]:
train_model(non_transfer_mobilenet, 10, swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:01<00:00, 28.83batch/s, loss=0.0317]

Model accuracy 0.9979101419448853, Loss 0.03167828315248092


In [81]:
test_loss, test_acc, confusion = validation(non_transfer_mobilenet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.20812787115573883, test loss 4.706421487357305
[[  0   1  29   0   0   0   0   0   0   0   0   0   0   0  28   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0]
 [  0  79 181   0   1   0   0   0   0   0   0   0   3   0 385   3   0   5
    0   0   0   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0
    0   0   7   0  13   0   0]
 [  0  60 213   0   0   0   0   0   0   0   0   0   3   0 366   1   0   0
    0   0   0   0   0   0   0   0   0   1   0   1   0   0   0   1   0   0
    0   0  11   0   9   0   0]
 [  0 130  68   0   0   0   0   0   0   0   0   0  28   0 179   0   0   4
    0   0   0   0   0   0   0   0   0   2   0   0   0   0   0   0   0   0
    0   0   1   0   4   0   0]
 [  0  21  92   0   3   0   0   0   0   0   0   0  14   0 447   3   0   0
    0   0   0   0   0   0   0   0   0   3   0   0   0   0   0   0   0   0
    0   0  17   0   4   0   0]
 [  0  75  57   0   0   0   0   0   0   0 

In [82]:
test_loss, test_acc, confusion = validation(non_transfer_mobilenet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9657701849937439, test loss 0.16902430140628263
[[20  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 30  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 12  0  0  0  0  1  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  7  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0 88  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0  1 32  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 15  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  7  0  0  0  2]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  1  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  0  0 52  0  0]
 [ 0  0  0  0  0  0  0  

In [83]:
non_transfer_Squeezenet = ModifiedSqueezeNet(43)

/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/img/Code/Python/Project/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [84]:
train_model(non_transfer_Squeezenet,10,swedish_training_loader)

Epoch 9: 100%|██████████| 30/30 [00:00<00:00, 40.87batch/s, loss=0.211]

Model accuracy 0.9383490085601807, Loss 0.2111475482583046


In [85]:
test_loss, test_acc, confusion = validation(non_transfer_Squeezenet, test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.16187722980976105, test loss 6.311029567666676
[[  0  15  10   0   0   0   0   0   0   0   0   0   0   0   0  17   0   4
    0   0   0   0   0   0   0   4   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   8   0   0]
 [  0 185 108   0   0   0   0   0   0   0   0   0   0  10  16 221   0  68
    0   0   0   0   0   0   0  20   0   3   3   0   0   0   0   4   0   1
    0   0   0   1  38   0   0]
 [  0  41 201   0   0   0   0   0   0   0   0   0   0  16  29 236   0  72
    0   0   0   0   0   0   0  17   0  18   0   0   0   0   0   3   0   0
    0   0   0   4  29   0   0]
 [  0  87   0   0   0   0   0   0   0   0   0   0   0   6  10 228   0  26
    0   0   0   0   0   0   0   5   0  43   1   0   0   0   0  10   0   0
    0   0   0   0   0   0   0]
 [  0  15  69   0   1   0   0   0   0   0   0   0   0   3   6 348   0  33
    0   0   0   0   0   0   0   0   0 117   0   0   0   0   0   6   0   2
    0   0   0   1   3   0   0]
 [  0  79   8   0   0   0   0   0   0   0 

In [86]:
test_loss, test_acc, confusion = validation(non_transfer_Squeezenet, swedish_test_loader, criterion)
print(f"Test accuracy {test_acc}, test loss {test_loss}")
def print_array(array):
  np.set_printoptions(threshold=np.inf)
  print(array)
  np.set_printoptions(threshold=1000)
print_array(confusion)

Test accuracy 0.9388753175735474, test loss 0.3075316832042657
[[19  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0]
 [ 1  8  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0 29  1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0 52  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0  3  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 17  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0 29  0  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  4  0  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 87  1  2  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  3  1 29  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  2  0 13  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  7  0  0  0  1]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  1]
 [ 0  0  0  0  1  0  0  0  0  0  0  0  0  1  0 50  0  1]
 [ 0  0  0  0  0  0  0  0